In [1]:
import win32com.client
import os
import pandas as pd

In [2]:
# Create a DataFrame to store the email log
email_log = pd.DataFrame(columns=['Serial', 'Sender', 'Receiver', 'Time', 'Subject', 'FileType', 'Attachment'])

In [3]:
# Access Outlook and retrieve the inbox folder
outlook = win32com.client.Dispatch("Outlook.Application")
mapi = outlook.GetNamespace("MAPI")
inbox = mapi.GetDefaultFolder(6)  # Inbox folder
messages = inbox.Items

In [4]:
serial_counter = 1  # Counter for serial numbers

In [5]:
# Create a folder to save the attachments
attachments_folder = r"D:\GIKI\CAA_intern\attachments"
os.makedirs(attachments_folder, exist_ok=True)

In [6]:
# Iterate over unread emails, download attachments, mark as read, and update the log
for message in messages:
    if message.Unread:
        sender = message.SenderEmailAddress
        receiver = message.ReceivedByName
        time = message.ReceivedTime
        subject = message.Subject

        attachments_info = []  # List to store attachment information
        
        for attachment in message.Attachments:
            filename = attachment.FileName
            file_extension = os.path.splitext(filename)[1]
            new_filename = f"{serial_counter}_{filename}"
            filepath = os.path.abspath(os.path.join(attachments_folder, new_filename))
            attachment.SaveAsFile(filepath)
            
            attachments_info.append({
                'FileName': new_filename,
                'FileType': file_extension
            })

        # Mark the email as read
        message.UnRead = False
        
        # Add email and attachment information to the log
        for attachment_info in attachments_info:
            email_log = email_log.append({                   # Append method 
                'Serial': serial_counter,
                'Sender': sender,
                'Receiver': receiver,
                'Time': str(time),
                'Subject': subject,
                'FileType': attachment_info['FileType'],
                'Attachment': attachment_info['FileName']
            }, ignore_index=True)                           
            
            serial_counter += 1

C:\Users\Owaise\AppData\Local\Temp\ipykernel_14064\2472139405.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  email_log = email_log.append({
C:\Users\Owaise\AppData\Local\Temp\ipykernel_14064\2472139405.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  email_log = email_log.append({
C:\Users\Owaise\AppData\Local\Temp\ipykernel_14064\2472139405.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  email_log = email_log.append({
C:\Users\Owaise\AppData\Local\Temp\ipykernel_14064\2472139405.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  email_log = email_log.append({


FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  email_log = email_log.append({

Therefore this is the alternative in case that ever happens.. But I find the above append method more easy..

In [9]:
"""
# Add email and attachment information to the log
for attachment_info in attachments_info:
    email_entry = {
        'Serial': serial_counter,
        'Sender': sender,
        'Receiver': receiver,
        'Time': str(time),
        'Subject': subject,
        'FileType': attachment_info['FileType'],
        'Attachment': attachment_info['FileName']
    }
    email_log = pd.concat([email_log, pd.DataFrame([email_entry])], ignore_index=True)
    serial_counter += 1

"""

"\n# Add email and attachment information to the log\nfor attachment_info in attachments_info:\n    email_entry = {\n        'Serial': serial_counter,\n        'Sender': sender,\n        'Receiver': receiver,\n        'Time': str(time),\n        'Subject': subject,\n        'FileType': attachment_info['FileType'],\n        'Attachment': attachment_info['FileName']\n    }\n    email_log = pd.concat([email_log, pd.DataFrame([email_entry])], ignore_index=True)\n    serial_counter += 1\n\n"

In [7]:
email_log

,Serial,Sender,Receiver,Time,Subject,FileType,Attachment
0,1,owais.sajid002@gmail.com,u2020337,2023-06-22 10:59:30.422000+00:00,attachment test 1,.pdf,1_2020337_intern_CV.pdf
1,2,owais.sajid002@gmail.com,u2020337,2023-06-22 11:00:33.318000+00:00,attachment test 2,.docx,2_CV_assignment3.docx
2,3,owais.sajid002@gmail.com,u2020337,2023-06-23 11:25:04.207000+00:00,Attachment test 3,.docx,3_CV_assignment3.docx
3,4,owais.sajid002@gmail.com,u2020337,2023-06-23 11:25:04.207000+00:00,Attachment test 3,.png,3_fiverr-logo.png


In [8]:
log_filepath = os.path.abspath(r"D:\GIKI\CAA_intern\email_log.xlsx")
email_log.to_excel(log_filepath, index=False)

### Sorting into seperate folders..

In [11]:
import os
import shutil

In [12]:
def sort_attachments_by_extension(attachments_folder):
    # Get all files in the attachments folder
    files = os.listdir(attachments_folder)
    
    # Create a dictionary to store folders for each extension
    extension_folders = {}
    
    # Iterate over the files and identify their extensions
    for file in files:
        file_extension = os.path.splitext(file)[1]
        
        # Create a folder for the extension if it doesn't exist
        if file_extension not in extension_folders:
            extension_folder = os.path.join(attachments_folder, file_extension)
            os.makedirs(extension_folder, exist_ok=True)
            extension_folders[file_extension] = extension_folder
        
        # Move the file to the respective extension folder
        file_path = os.path.join(attachments_folder, file)
        destination_folder = extension_folders[file_extension]
        shutil.move(file_path, destination_folder)

In [13]:
sort_attachments_by_extension(attachments_folder)